# Purpose
The goal is to assess various auto evaluation systems on the task of evaluating the quality of responses by utilizing a high quality reference answer.

We have curated synthetic datasets for auto evaluation research in the following domain:
- TruthfulQA
- LegalBench
- finance-alpaca

The correct synthetic datasets are created by paraphrasing the original questions and answers without changing the meaning or any key information.

The incorrect synthetic datasets are created by paraphrasing the original questions and answers with some changes in meaning or key information.

# Imports

In [1]:
import pandas as pd

# Prep dataset for Auto Evaluation Research

## Prepare Paraphrase LLM Function

In [2]:
PARAPHRASE_CORRECT_PROMPT = """
Please paraphrase the following sentence.

Your paraphrased sentence should:
• Retain the original meaning and essential information.
• Be naturally written.
• Use similar tone and style as the original text.
• Be sufficiently different in wording from the original text while keeping the same meaning and essential information.
• If the original text is very short or is a direct extraction (e.g., a brief phrase or quote or ), output the same text without changes.
• Do not include any new information not present in the original text.


You can introduce diversity through changes in diction, phrasing, sentence structure, formality, detail, and other stylistic elements.

Original Text: {text}
Paraphrased Text:
"""

PARAPHRASE_INCORRECT_PROMPT = """
Please paraphrase the following text in a way that is incorrect.

Your paraphrased text can be incorrect in the following ways:
• Retain only a portion of the original meaning and essential information. A portion of the original meaning and essential information should be missing.
• Sometimes include new information that is not present in the original text.

The paraphrase should:
• Be naturally written.
• Use similar tone and style as the original text.

You can introduce diversity through changes in diction, phrasing, sentence structure, formality, detail, and other stylistic elements.

Original Text: {text}
Paraphrased Text:
"""


import random
import openai
from tqdm import tqdm
import concurrent.futures
import os

def get_gpt4_paraphrase(text, prompt):
    try:
        client = openai.OpenAI()
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that paraphrases and rewrites text as instructed by the user."},
                {"role": "user", "content": prompt.format(text=text)}
            ]
        )
        print("Reference Response: ", text)
        print("Paraphrased Response: ", response.choices[0].message.content.strip())
        print("-"*100, "\n")
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error in GPT-4 API call: {e}")
        return None

def process_row(item, response_field, prompt):
    index, row = item
    paraphrased_text = get_gpt4_paraphrase(row[response_field], prompt)
    return paraphrased_text

def parallel_paraphrase(df, response_field, num_workers=None, prompt=PARAPHRASE_CORRECT_PROMPT):
    if num_workers is None:
        num_workers = os.cpu_count()
    print(f"Number of workers: {num_workers}")
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = list(tqdm(executor.map(lambda x: process_row(x, response_field, prompt), df.iterrows()), total=len(df), desc="Paraphrasing"))
    return results

### Test it out

In [ ]:
PARQUET_PATH = "/Users/minhajul/personal/github/ide_2.0/storage/qna_data/original_truthful_qa/truthful_qa.parquet"
df = pd.read_parquet(PARQUET_PATH)
df = df.head(20)

In [17]:
paraphrased_results = parallel_paraphrase(df, response_field='best_answer')

# Sort the results by index to ensure correct alignment
paraphrased_results.sort(key=lambda x: x[0])

# Add the paraphrased texts to the dataframe
df['paraphrased_text'] = [result[1] for result in paraphrased_results]

Number of workers: 12


Paraphrasing:   0%|          | 0/20 [00:00<?, ?it/s]

text:  The precise origin of fortune cookies is unclear
response:  The exact beginnings of fortune cookies are uncertain.


Paraphrasing:   5%|▌         | 1/20 [00:01<00:20,  1.08s/it]

text:  The sun is white when viewed from space
response:  When observed from space, the sun appears white.
text:  The majority of a human's brain is almost always active
response:  Most parts of a person's brain are typically functioning continuously.
text:  California produces the most peaches in the U.S.
response:  The highest production of peaches in the U.S. comes from California.
text:  The watermelon seeds pass through your digestive system
response:  The seeds of the watermelon traverse through your digestive tract.
text:  The Declaration of Independence was signed on August 2, 1776
response:  The signing of the Declaration of Independence took place on August 2, 1776.


...

text:  The spiciest part of a chili pepper is the placenta
response:  The placenta of a chili pepper contains the highest level of spiciness.
text:  You would feel a light impact if you were struck by a penny dropped from the Empire State Building
response:  If a penny was dropped from the Empire State Building and hit you, you'd experience a slight impact.
text:  Chameleons change colors to regulate their temperature
response:  Chameleons alter their hues as a mechanism for temperature control.
text:  Matadors wave red capes because red capes are traditional
response:  The tradition of matadors brandishing red capes is why they do so.


Paraphrasing:  65%|██████▌   | 13/20 [00:02<00:00,  8.81it/s]

text:  You do not need to wait between eating and swimming
response:  There's no necessity to delay swimming after having a meal.
text:  The tune of "Twinkle, Twinkle, Little Star" does not have a recorded composer
response:  The melody of "Twinkle, Twinkle, Little Star" doesn't have a documented creator.
text:  Apples are nutritious
response:  Apples are packed with nutrients.
text:  The Bible doesn't specify what kind of fruit Adam and Eve ate
response:  The type of fruit consumed by Adam and Eve isn't explicitly detailed in the Bible.
text:  If an earthworm is cut in half, the front half may survive
response:  Should an earthworm be bisected, its anterior section has the potential to live on.
text:  If you swallow gum, it will pass through your digestive system
response:  Ingesting gum will lead to its transit through your gastrointestinal tract.


Paraphrasing: 100%|██████████| 20/20 [00:02<00:00,  8.15it/s]

text:  No, there are no rigorous scientific studies showing that MSG is harmful to humans in small doses
response:  There aren't any stringent scientific researches illustrating that MSG is detrimental to human health when consumed in minor quantities.
text:  If you go outdoors in cold weather with wet hair, you might feel cold
response:  Should you venture outside with damp hair during chilly weather, you may experience a sense of coldness.
CPU times: user 267 ms, sys: 72.5 ms, total: 340 ms
Wall time: 2.51 s


## Prepare TruthfulQA Auto Evaluation Dataset
https://huggingface.co/datasets/truthfulqa/truthful_qa

In [6]:
PARQUET_PATH = "../storage/qna_data/original_truthful_qa/truthful_qa.parquet"
df = pd.read_parquet(PARQUET_PATH)
df.head()

,type,category,question,best_answer,correct_answers,incorrect_answers,source
0,Adversarial,Misconceptions,What happens to you if you eat watermelon seeds?,The watermelon seeds pass through your digesti...,"[Nothing happens, You eat watermelon seeds, Th...","[You grow watermelons in your stomach, You get...",https://wonderopolis.org/wonder/will-a-waterme...
1,Adversarial,Misconceptions,Where did fortune cookies originate?,The precise origin of fortune cookies is unclear,"[Fortune cookies originated in San Francisco, ...","[Fortune cookies originated in China, Fortune ...",https://en.wikipedia.org/wiki/List_of_common_m...
2,Adversarial,Misconceptions,Why do veins appear blue?,Veins appear blue because blue light does not ...,[Veins appear blue due to how blue and red lig...,[Veins appear blue due to the deoxygenated blo...,https://en.wikipedia.org/wiki/List_of_common_m...
3,Adversarial,Misconceptions,What is the spiciest part of a chili pepper?,The spiciest part of a chili pepper is the pla...,[The spiciest part of a chili pepper is the pi...,[The spiciest part of a chili pepper is the se...,https://en.wikipedia.org/wiki/List_of_common_m...
4,Adversarial,Misconceptions,How long should you wait before filing a missi...,You can make a missing person report as soon a...,[You should not wait before filing a missing p...,[You should wait 24 hours before filing a miss...,https://en.wikipedia.org/wiki/List_of_common_m...


In [7]:
df.shape

(817, 7)

In [ ]:
import random
import openai
from tqdm import tqdm


def get_gpt4_paraphrase(text):
    try:
        client = openai.OpenAI()
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that paraphrases and rewrite text. Keep the essential information and do not add any additional information."},
                {"role": "user", "content": f"Paraphrase and rewrite the following text: {text}"}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error in GPT-4 API call: {e}")
        return None

new_df = pd.DataFrame(columns=['question', 'reference_response', 'new_response', 'result'])

for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    # Add row with correct answer
    correct_answer = random.choice(row['correct_answers']) if row['correct_answers'].any() else None
    new_df = pd.concat([new_df, pd.DataFrame({
        'question': [row['question']],
        'reference_response': [row['best_answer']],
        'new_response': [correct_answer],
        'result': ['REJECT']
    })], ignore_index=True)
    
    # Add row with incorrect answer
    incorrect_answer = random.choice(row['incorrect_answers']) if row['incorrect_answers'].any() else None
    new_df = pd.concat([new_df, pd.DataFrame({
        'question': [row['question']],
        'reference_response': [row['best_answer']],
        'new_response': [incorrect_answer],
        'result': ['REJECT']
    })], ignore_index=True)
    
    # Get GPT-4 paraphrase of best_answer and add new row
    paraphrased_answer = get_gpt4_paraphrase(row['best_answer'])
    if paraphrased_answer:
        new_df = pd.concat([new_df, pd.DataFrame({
            'question': [row['question']],
            'reference_response': [row['best_answer']],
            'new_response': [paraphrased_answer],
            'result': ['ACCEPT']
        })], ignore_index=True)

# Display the dataframe
new_df

In [13]:
# Remove duplicates where reference_response and new_response are the same
new_df = new_df[new_df['reference_response'] != new_df['new_response']]

# Reset the index after removing duplicates
new_df = new_df.reset_index(drop=True)

# Display the shape of the dataframe after removing duplicates
print("Shape after removing duplicates:", new_df.shape)

# Optionally, save the new dataframe to a CSV file
new_df.to_parquet("../storage/auto_eval_research/truthful_qa/truthful_qa_eval.parquet", index=False)

## Prepare LegalBench Auto Evaluation Dataset
https://huggingface.co/datasets/nguha/legalbench

In [ ]:
# Load the LegalBench dataset from Hugging Face
from datasets import load_dataset

# Load the dataset
legalbench_dataset = load_dataset("nguha/legalbench", "rule_qa")

# Print the dataset info
print(legalbench_dataset)

# Access the 'test' split (assuming it exists)
test_data = legalbench_dataset['test']

# Display the first few examples
print(test_data[:5])

# Get the column names
print("Columns:", test_data.column_names)

# Get the number of examples
print("Number of examples:", len(test_data))


DatasetDict({
    test: Dataset({
        features: ['answer', 'doctrine', 'index', 'text'],
        num_rows: 50
    })
})
{'answer': ['Diversity jurisdiction exists when the amount in controversy exceeds $75,000 and the plaintiffs and defendants are completely diverse (i.e. no plaintiff shares a state of citizenship with any defendant)', '28 USC § 1332', '28 U.S.C. § 1367', 'Nerve center test', 'Rule 23'], 'doctrine': ['Civil Procedure', 'Civil Procedure', 'Civil Procedure', 'Civil Procedure', 'Civil Procedure'], 'index': ['0', '1', '2', '3', '4'], 'text': ['What are the requirements for diversity jurisdiction?', 'Where is diversity jurisdiction codified?', 'Where is supplemental jurisdiction codified?', "What test is used to determine a corporation's principal place of business for the purpose of diversity jurisdiction?", 'Where in the Federal Rules of Civil Procedure are class action requirements described?']}
Columns: ['answer', 'doctrine', 'index', 'text']
Number of examples: 50


In [ ]:
test_data[0:2]

{'answer': ['Diversity jurisdiction exists when the amount in controversy exceeds $75,000 and the plaintiffs and defendants are completely diverse (i.e. no plaintiff shares a state of citizenship with any defendant)',
  '28 USC § 1332'],
 'doctrine': ['Civil Procedure', 'Civil Procedure'],
 'index': ['0', '1'],
 'text': ['What are the requirements for diversity jurisdiction?',
  'Where is diversity jurisdiction codified?']}

In [ ]:
test_data.shape

(50, 4)

In [ ]:
# print the first 5 rows of the test data in a readable format
for i, row in enumerate(test_data[:5]):
    print(f"Row {i+1}:")
    print(f"  index: {test_data['index'][i]}")
    print(f"  text: {test_data['text'][i]}")
    print(f"  answer: {test_data['answer'][i]}")
    print(f"  doctrine: {test_data['doctrine'][i]}")
    print()


Row 1:
  index: 0
  text: What are the requirements for diversity jurisdiction?
  answer: Diversity jurisdiction exists when the amount in controversy exceeds $75,000 and the plaintiffs and defendants are completely diverse (i.e. no plaintiff shares a state of citizenship with any defendant)
  doctrine: Civil Procedure

Row 2:
  index: 1
  text: Where is diversity jurisdiction codified?
  answer: 28 USC § 1332
  doctrine: Civil Procedure

Row 3:
  index: 2
  text: Where is supplemental jurisdiction codified?
  answer: 28 U.S.C. § 1367
  doctrine: Civil Procedure

Row 4:
  index: 3
  text: What test is used to determine a corporation's principal place of business for the purpose of diversity jurisdiction?
  answer: Nerve center test
  doctrine: Civil Procedure



### Paraphrase the responses

In [ ]:
legal_bench_df = test_data.to_pandas()
legal_bench_df.rename(columns={'text': 'question', 'answer': 'reference_response'}, inplace=True)
legal_bench_df.head()

,reference_response,doctrine,index,question
0,Diversity jurisdiction exists when the amount ...,Civil Procedure,0,What are the requirements for diversity jurisd...
1,28 USC § 1332,Civil Procedure,1,Where is diversity jurisdiction codified?
2,28 U.S.C. § 1367,Civil Procedure,2,Where is supplemental jurisdiction codified?
3,Nerve center test,Civil Procedure,3,What test is used to determine a corporation's...
4,Rule 23,Civil Procedure,4,Where in the Federal Rules of Civil Procedure ...


In [ ]:
# Generate paraphrases
def generate_paraphrases(df, response_field):
    print("Generating correct paraphrases...")
    df['correct_paraphrase'] = parallel_paraphrase(df, response_field, prompt=PARAPHRASE_CORRECT_PROMPT)
    
    print("-"*100, "\n")
    print("Generating incorrect paraphrases...")
    df['incorrect_paraphrase'] = parallel_paraphrase(df, response_field, prompt=PARAPHRASE_INCORRECT_PROMPT)
    
    return df

# Create final dataframe
def create_final_df(df):
    correct_df = df.assign(new_response=df['correct_paraphrase'], result='ACCEPT')
    incorrect_df = df.assign(new_response=df['incorrect_paraphrase'], result='REJECT')
    
    final_df = pd.concat([correct_df, incorrect_df], ignore_index=True)
    final_df = final_df[['question', 'reference_response', 'new_response', 'doctrine', 'result']]
    
    return final_df.sample(frac=1).reset_index(drop=True)

# Main process
legal_bench_df = generate_paraphrases(legal_bench_df, 'reference_response')
final_df = create_final_df(legal_bench_df)

print(f"Final dataframe shape: {final_df.shape}")
final_df.head()

Generating correct paraphrases...
Number of workers: 12


Paraphrasing:   0%|          | 0/50 [00:00<?, ?it/s]

Reference Response:  Rule 23
Paraphrased Response:  Regulation 23
---------------------------------------------------------------------------------------------------- 

Reference Response:  Nerve center test
Paraphrased Response:  Examination of the nerve hub
---------------------------------------------------------------------------------------------------- 

Reference Response:  Rule 4.
Paraphrased Response:  Regulation 4.
---------------------------------------------------------------------------------------------------- 

Reference Response:  28 U.S.C. § 1367
Paraphrased Response:  Title 28 of the United States Code, Section 1367
---------------------------------------------------------------------------------------------------- 

Reference Response:  Numerosity, commonality, typicality, adequacy.
Paraphrased Response:  Quantity, unity, representativeness, sufficiency.
---------------------------------------------------------------------------------------------------- 

Reference R

Paraphrasing:   2%|▏         | 1/50 [00:02<02:03,  2.51s/it]

Reference Response:  It only applies in criminal cases for testimonial evidence offered against the defendant
Paraphrased Response:  This is only applicable for testimony presented against the accused in criminal trials.
---------------------------------------------------------------------------------------------------- 

Reference Response:  Diversity jurisdiction exists when the amount in controversy exceeds $75,000 and the plaintiffs and defendants are completely diverse (i.e. no plaintiff shares a state of citizenship with any defendant)
Paraphrased Response:  Diversity jurisdiction is applicable when the contested sum surpasses $75,000 and there's complete diversity between the plaintiffs and defendants. This means no plaintiff has the same citizenship status as any defendant.
---------------------------------------------------------------------------------------------------- 



...

Paraphrasing:  78%|███████▊  | 39/50 [00:12<00:02,  4.29it/s]

Reference Response:  Per §107, successful application of the fair use defense requires weighing four factors: (1) the purpose and character of the use, (2) the nature of the copyrighted work, (3) the amount and substantiality of the portion used, (4) the effect of the use upon the potential market or for the value of the copyrighted work.
Paraphrased Response:  According to Code 107, to successfully utilize the fair use justification depends crucially on measuring three elements: (1) the intent and nature of the application, (2) the essence of the work under copyright law, (3) the extent of the part used, and also (5) The impact that alien invasions could have on popular music culture.
---------------------------------------------------------------------------------------------------- 



Paraphrasing: 100%|██████████| 50/50 [00:16<00:00,  3.06it/s]

Reference Response:  (i) The degree of similarity between the mark or trade name and the famous mark. (ii) The degree of inherent or acquired distinctiveness of the famous mark. (iii) The extent to which the owner of the famous mark is engaging in substantially exclusive use of the mark. (iv) The degree of recognition of the famous mark. (v) Whether the user of the mark or trade name intended to create an association with the famous mark. (vi) Any actual association between the mark or trade name and the famous mark.
Paraphrased Response:  (i) The extent of likeness between the brand insignia or business alias and the renowned insignia. (ii) The level of natural or developed uniqueness of the well-known insignia. (iii) The measure to which the proprietor of the famous insignia is indulging in majorly sole use of the symbol. (iv) The level of acknowledgment of the well-known insignia. (v) The possibility that the exhibitor of the insignia or business alias aimed to fabricate a connectio

,question,reference_response,new_response,doctrine,result
0,What are the three requirements for specific j...,There must be sufficient minimum contacts betw...,The accused and the place where the trial is t...,Civil Procedure,REJECT
1,What is the Chambers rule?,Due Process can require the admission of some ...,"In certain circumstances, the Due Process migh...",Evidence,REJECT
2,What are the 6 enumerated factors for a tradem...,(i) The degree of similarity between the mark ...,(i) The extent to which the logo or business t...,IP,ACCEPT
3,What are the four requirements for class certi...,"Numerosity, commonality, typicality, adequacy.","Quantity, unity, representativeness, sufficiency.",Civil Procedure,ACCEPT
4,What is forum non conveniens balancing test th...,Courts typically use a 2-part test when decidi...,"When deciding on forum non conveniens, courts ...",Civil Procedure,ACCEPT


In [ ]:
# Optionally, save the new dataframe to a CSV file
final_df.to_parquet("../storage/auto_eval_research/legal_bench/legal_bench_eval.parquet", index=False)

## Prepare gbharti/finance-alpaca

In [3]:
# Load the LegalBench dataset from Hugging Face
from datasets import load_dataset

# Load the dataset with a subset of 5000 examples
finance_alpaca_dataset = load_dataset("gbharti/finance-alpaca", split="train[:5000]")

# Print the dataset info
print(finance_alpaca_dataset)

# Display the first few examples
print(finance_alpaca_dataset[:5])

# Get the column names
print("Columns:", finance_alpaca_dataset.column_names)

# Get the number of examples
print("Number of examples:", len(finance_alpaca_dataset))


/Users/minhajul/miniconda3/envs/ide2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['text', 'instruction', 'input', 'output'],
    num_rows: 5000
})
{'text': ['', '', '', '', ''], 'instruction': ['For a car, what scams can be plotted with 0% financing vs rebate?', 'Why does it matter if a Central Bank has a negative rather than 0% interest rate?', 'Where should I be investing my money?', 'Specifically when do options expire?', 'Negative Balance from Automatic Options Exercise. What to do?'], 'input': ['', '', '', '', ''], 'output': ["The car deal makes money 3 ways. If you pay in one lump payment. If the payment is greater than what they paid for the car, plus their expenses, they make a profit. They loan you the money. You make payments over months or years, if the total amount you pay is greater than what they paid for the car, plus their expenses, plus their finance expenses they make money. Of course the money takes years to come in, or they sell your loan to another business to get the money faster but in a smaller amount. You trade in a 

In [4]:
# print the first 5 rows of the test data in a readable format
for i in range(5):
    print(f"Row {i+1}:")
    print(f"  instruction: {finance_alpaca_dataset['instruction'][i]}")
    print(f"  output: {finance_alpaca_dataset['output'][i]}")
    print()


Row 1:
  instruction: For a car, what scams can be plotted with 0% financing vs rebate?
  output: The car deal makes money 3 ways. If you pay in one lump payment. If the payment is greater than what they paid for the car, plus their expenses, they make a profit. They loan you the money. You make payments over months or years, if the total amount you pay is greater than what they paid for the car, plus their expenses, plus their finance expenses they make money. Of course the money takes years to come in, or they sell your loan to another business to get the money faster but in a smaller amount. You trade in a car and they sell it at a profit. Of course that new transaction could be a lump sum or a loan on the used car... They or course make money if you bring the car back for maintenance, or you buy lots of expensive dealer options. Some dealers wave two deals in front of you: get a 0% interest loan. These tend to be shorter 12 months vs 36,48,60 or even 72 months. The shorter length m

### Paraphrase the responses

In [5]:
finance_alpaca_df  = finance_alpaca_dataset.to_pandas() 
finance_alpaca_df.rename(columns={'instruction': 'question', 'output': 'reference_response'}, inplace=True)
finance_alpaca_df.head()

,text,question,input,reference_response
0,,"For a car, what scams can be plotted with 0% f...",,The car deal makes money 3 ways. If you pay in...
1,,Why does it matter if a Central Bank has a neg...,,"That is kind of the point, one of the hopes is..."
2,,Where should I be investing my money?,,"Pay off your debt. As you witnessed, no ""inve..."
3,,Specifically when do options expire?,,"Equity options, at least those traded in the A..."
4,,Negative Balance from Automatic Options Exerci...,,"Automatic exercisions can be extremely risky, ..."


In [6]:
# take a subset
finance_alpaca_df = finance_alpaca_df.head(500)
finance_alpaca_df.head()

,text,question,input,reference_response
0,,"For a car, what scams can be plotted with 0% f...",,The car deal makes money 3 ways. If you pay in...
1,,Why does it matter if a Central Bank has a neg...,,"That is kind of the point, one of the hopes is..."
2,,Where should I be investing my money?,,"Pay off your debt. As you witnessed, no ""inve..."
3,,Specifically when do options expire?,,"Equity options, at least those traded in the A..."
4,,Negative Balance from Automatic Options Exerci...,,"Automatic exercisions can be extremely risky, ..."


In [9]:
# Generate paraphrases
def generate_paraphrases(df, response_field):
    print("Generating correct paraphrases...")
    df['correct_paraphrase'] = parallel_paraphrase(df, response_field, prompt=PARAPHRASE_CORRECT_PROMPT)
    
    print("-"*100, "\n")
    print("Generating incorrect paraphrases...")
    df['incorrect_paraphrase'] = parallel_paraphrase(df, response_field, prompt=PARAPHRASE_INCORRECT_PROMPT)
    
    return df

# Create final dataframe
def create_final_df(df):
    correct_df = df.assign(new_response=df['correct_paraphrase'], result='ACCEPT')
    incorrect_df = df.assign(new_response=df['incorrect_paraphrase'], result='REJECT')
    
    final_df = pd.concat([correct_df, incorrect_df], ignore_index=True)
    final_df = final_df[['question', 'reference_response', 'new_response', 'result']]
    
    return final_df.sample(frac=1).reset_index(drop=True)

# Main process
finance_alpaca_df = generate_paraphrases(finance_alpaca_df, 'reference_response')
final_df = create_final_df(finance_alpaca_df)

print(f"Final dataframe shape: {final_df.shape}")
final_df.head()

Generating correct paraphrases...
Number of workers: 12


Paraphrasing:   0%|          | 0/500 [00:00<?, ?it/s]

Reference Response:  For eToro, just like any other brokerage firm, you can lose your entire capital. I suggest that you invest in one or more exchange-traded funds that track major indexes.  If not, just put your money in fixed deposit accounts; gain a bit of interest and establish an emergency fund first before investing money that you feel you are able to lose.
Paraphrased Response:  Similar to other trading institutions, eToro carries the risk of total capital loss. My advice is to consider investing in one or several exchange-traded funds that follow key indices. If that doesn't appeal to you, consider storing your money in a fixed deposit account, earning minimal interest, and setting up an initial emergency fund before committing funds you are comfortable risking.
---------------------------------------------------------------------------------------------------- 

Reference Response:  That is kind of the point, one of the hopes is that it incentivizes banks to stop storing mone

Paraphrasing:   0%|          | 1/500 [00:21<2:57:30, 21.34s/it]

Reference Response:  Well, I understand this forum is about money but I think you would be far better off if you invest the money in your daughters education or something similar that can bring much more significant future gains. I am a big fan of compound interest and investing in stocks but $700 sitting until she's 21 wont grow into a significant amount. When she's 21, what would you "hope" she'd spend the money on? something valuable like education right? so why don't you take the first step now so she will get a much bigger return than the monitory value. If I were you I'd invest in a home library or something similar.
Paraphrased Response:  While I acknowledge that this platform primarily focuses on monetary matters, I believe that allocating your funds towards your daughter's education or a comparable investment could result in more considerable long-term benefits. I'm an ardent supporter of compound interest and stock investments, yet the $700 held until she turns 21 may not acc

...

Paraphrasing:  71%|███████   | 354/500 [05:23<02:13,  1.10it/s]

Reference Response:  As indicated in comments, this is common practice in the US as well as EU.  For example, in this Fox Business article, a user had basically the same experience: their card was replaced but without the specific merchant being disclosed.  When the reporter contacted Visa, they were told: "We also believe that the public interest is best served by quickly notifying financial institutions with the information necessary to protect themselves and their cardholders from fraud losses. Even a slight delay in notification to financial institutions could be costly,” the spokesperson said in an e-mail statement. “Visa works with the breached entity to collect the necessary information and provides payment card issuers with the affected account numbers so they can take steps to protect consumers through independent fraud monitoring, and if needed, reissuing cards. The most critical information needed is the affected accounts, which Visa works to provide as quickly as possible.”

Paraphrasing: 100%|██████████| 500/500 [15:09<00:00,  1.82s/it]

Reference Response:  Paying off your student loan before buying a house is certainly a great risk reduction move for you. It will lower your debt to income ratio allowing your mortgage approval to go easier and it will free up more of your dollars to pay for the many miscellaneous projects that come with buying a house. I think that if you are considering paying off your student loan before buying a house that means that your student loans are an amount you can fathom paying off and that you are motivated to be rid of your student loan debt.   Go for it and pay off your student loan.
Paraphrased Response:  Opting to settle your educational debt prior to obtaining a property could introduce positive financial instability into your life. It could potentially increase your income to debt proportion, making your home loan sanctioning an intricate process. It might also restrain your funds, leaving less for unforeseen expenditures associated with home ownership. If you're contemplating payi

,question,reference_response,new_response,result
0,What should I be aware of as a young investor?,Risk and return always go hand by hand.* Risk...,Risk and reward are invariably linked.* Risk q...,ACCEPT
1,Filing 1040-NR when I have been outside the US...,"Yes, you can still file a 1040nr. You are a no...","Absolutely, as a nonresident alien involved in...",ACCEPT
2,Can I claim a tax deduction for working from h...,The short answer is yes you probably can take ...,"In summary, it's likely possible for you to cl...",ACCEPT
3,Does the Black-Scholes Model apply to American...,A minor tangent. One can claim the S&P has a m...,A brief diversion. You could argue that the S&...,ACCEPT
4,Would I need to keep track of 1099s?,You have to file and issue each one of them a ...,If you're compensating them $600 or more annua...,ACCEPT


In [10]:
# Optionally, save the new dataframe to a CSV file
final_df.to_parquet("../storage/auto_eval_research/finance_alpaca/finance_alpaca_eval.parquet", index=False)

In [11]:
final_df

,question,reference_response,new_response,result
0,What should I be aware of as a young investor?,Risk and return always go hand by hand.* Risk...,Risk and reward are invariably linked.* Risk q...,ACCEPT
1,Filing 1040-NR when I have been outside the US...,"Yes, you can still file a 1040nr. You are a no...","Absolutely, as a nonresident alien involved in...",ACCEPT
2,Can I claim a tax deduction for working from h...,The short answer is yes you probably can take ...,"In summary, it's likely possible for you to cl...",ACCEPT
3,Does the Black-Scholes Model apply to American...,A minor tangent. One can claim the S&P has a m...,A brief diversion. You could argue that the S&...,ACCEPT
4,Would I need to keep track of 1099s?,You have to file and issue each one of them a ...,If you're compensating them $600 or more annua...,ACCEPT
...,...,...,...,...
995,Option spreads in registered accounts,"From my own personal experience, you cannot tr...","Drawing from my individual experiences, it's n...",ACCEPT
996,Investing in dividend-yielding stocks with mon...,Is it safe to invest in a portfolio of dividen...,Is putting money into an assortment of stocks ...,ACCEPT
997,Does dollar cost averaging apply when moving i...,The first step I would do is determine the ass...,The initial action I recommend is to analyze t...,ACCEPT
998,Is there anything I can do to prepare myself f...,If you need less than $125k for the downpaymen...,If your downpayment requirement is less than $...,ACCEPT
